In [2]:
!pip install -q -U trax

     |████████████████████████████████| 368kB 4.7MB/s 
     |████████████████████████████████| 1.5MB 15.6MB/s 
     |████████████████████████████████| 2.6MB 33.4MB/s 
     |████████████████████████████████| 163kB 50.7MB/s 
     |████████████████████████████████| 194kB 55.3MB/s 
     |████████████████████████████████| 983kB 53.8MB/s 
     |████████████████████████████████| 358kB 50.7MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 368kB 33.5MB/s 
     |████████████████████████████████| 655kB 51.1MB/s 
     |████████████████████████████████| 5.3MB 21.6MB/s 
     |████████████████████████████████| 307kB 53.1MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
     |████████████████████████████████| 1.1MB 50.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 3.5MB 49.8MB/s 
     |████████████████████████████████| 235kB 50.4MB/s 
     |████████████████████████████████| 51kB 7.0MB/s

In [3]:
import trax
import trax.fastmath.numpy as fastnp
from trax import layers as tl
from trax.supervised import training
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('questions.csv',usecols = ['question1','question2','is_duplicate'])
df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
df_dup = df[df.is_duplicate == 1]
df_dup.head()

,question1,question2,is_duplicate
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,How can I be a good geologist?,What should I do to be a great geologist?,1
11,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,What was your first sexual experience like?,What was your first sexual experience?,1


In [6]:
list1 = [str(q) for q in df_dup.question1]
list1[0:5]

['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
 'How can I be a good geologist?',
 'How do I read and find my YouTube comments?',
 'What can make Physics easy to learn?',
 'What was your first sexual experience like?']

In [7]:
list2 = [str(q) for q in df_dup.question2]
list2[:5]

["I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",
 'What should I do to be a great geologist?',
 'How can I see all my Youtube comments?',
 'How can you make physics easy to learn?',
 'What was your first sexual experience?']

In [8]:
vocab = {'__UNK__':0,'<PAD>':1}
f_list = list1+list2
for sentence in f_list:
  for word in nltk.word_tokenize(sentence.lower()):
    if word not in vocab:
      vocab[word] = len(vocab)

In [9]:
len(vocab)

33759

In [10]:
vocab.keys()

dict_keys(['__UNK__', '<PAD>', 'astrology', ':', 'i', 'am', 'a', 'capricorn', 'sun', 'cap', 'moon', 'and', 'rising', '...', 'what', 'does', 'that', 'say', 'about', 'me', '?', 'how', 'can', 'be', 'good', 'geologist', 'do', 'read', 'find', 'my', 'youtube', 'comments', 'make', 'physics', 'easy', 'to', 'learn', 'was', 'your', 'first', 'sexual', 'experience', 'like', 'would', 'trump', 'presidency', 'mean', 'for', 'current', 'international', 'master', '’', 's', 'students', 'on', 'an', 'f1', 'visa', 'manipulation', 'why', 'are', 'so', 'many', 'quora', 'users', 'posting', 'questions', 'readily', 'answered', 'google', 'rockets', 'look', 'white', 'should', 'prepare', 'ca', 'final', 'law', 'some', 'special', 'cares', 'someone', 'with', 'nose', 'gets', 'stuffy', 'during', 'the', 'night', 'game', 'of', 'thrones', 'villain', 'most', 'likely', 'give', 'you', 'mercy', 'we', 'upsc', 'examples', 'products', 'from', 'crude', 'oil', 'friends', '.', 'is', 'career', 'launcher', 'rbi', 'grade', 'b', 'prepara

In [11]:
def text_to_tensor(sentence,unk = '__UNK__'):
  unknown = vocab[unk]
  tensor = []
  for word in nltk.word_tokenize(sentence.lower()):
    tensor.append(vocab.get(word,unknown))
  return tensor

In [12]:
#test
sent = "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"
print(text_to_tensor(sent))

[4, 956, 6, 3678, 7, 256, 8, 122, 10, 11, 17827, 159, 7, 258, 14, 15, 358, 17, 18, 19, 20]


In [13]:
length = int(len(list1)*0.8)
train_q1 = list1[:length]
train_q2 = list2[:length]
val_q1 = list1[length:]
val_q2 = list2[length:]
len(train_q1),len(train_q2),len(val_q1),len(val_q2)

(119444, 119444, 29862, 29862)

In [14]:
import random
def dataGenerator(l1,l2,batch_size,pad,shuffle = True):
  index = 0
  len_lines = len(l1)
  index_lines = list(range(len_lines))
  if shuffle:
    random.shuffle(index_lines)
  x1 = []
  x2 = []
  while True:
    if index >= len_lines:
      index = 0
      if shuffle:
        random.shuffle(index_lines)
    q1 = l1[index_lines[index]]
    #print(q1)
    x1.append(text_to_tensor(q1))
    q2 = l2[index_lines[index]]
    #print(q2)
    x2.append(text_to_tensor(q2))
    index += 1

    if batch_size == len(x1): 
      max_len = max(max([len(x) for x in x1]),max([len(x) for x in x2]))
      batch_x1 = []
      batch_x2 = []
      for i in range(batch_size):
        question1 = x1[i] + [pad]*(max_len-len(x1[i]))
        question2 = x2[i] + [pad]*(max_len-len(x2[i]))
        batch_x1.append(question1)
        batch_x2.append(question2)
      final_q1 = fastnp.array(batch_x1)
      final_q2 = fastnp.array(batch_x2)
      yield final_q1,final_q2
      x1 = []
      x2 = []




In [15]:
#test
train = next(dataGenerator(train_q1,train_q2,4,vocab['<PAD>']))
train

(DeviceArray([[  21,   22,    4,  579, 2628, 4112,  159,   29, 2252,   20,
                  1,    1,    1,    1,    1,    1],
              [  59,   60,  157, 2200,  408,   20,    1,    1,    1,    1,
                  1,    1,    1,    1,    1,    1],
              [  21,   26,    4,   74,  534, 4797,   54,    6,   75,   76,
                 20,    1,    1,    1,    1,    1],
              [  21,   22,  592, 3458, 1050, 4692,   20,    1,    1,    1,
                  1,    1,    1,    1,    1,    1]], dtype=int32),
 DeviceArray([[   21,    26,     4,   579,  2628,  4112,    20,     1,
                   1,     1,     1,     1,     1,     1,     1,     1],
              [   21,   107,   157,   242,     6,  8271,   408,    20,
                   1,     1,     1,     1,     1,     1,     1,     1],
              [   14,   107,    87,   154,   288,    35,    74,    47,
                  75,    76,    77,    11, 13393,    20,     1,     1],
              [  107,   221,     6,  3458,    47

In [16]:
import itertools
batch_size = 64
train_generator = itertools.cycle(dataGenerator(train_q1,train_q2,batch_size,vocab['<PAD>']))
eval_generator = itertools.cycle(dataGenerator(val_q1,val_q2,batch_size,vocab['<PAD>']))

In [17]:
def siamese_model(vocab_size=33759,d_model = 256):
  def normalize(x):
    return x/fastnp.sqrt(fastnp.sum(x*x,axis = -1,keepdims=True))
  
  model = tl.Serial(tl.Embedding(vocab_size,d_model),tl.LSTM(n_units=d_model),tl.Mean(axis=-1),tl.Fn('Normalize',lambda x : normalize(x)))
  return tl.Parallel(model,model)

In [18]:
display(siamese_model())

Parallel_in2_out2[
  Serial[
    Embedding_33759_256
    LSTM_256
    Mean
    Normalize
  ]
  Serial[
    Embedding_33759_256
    LSTM_256
    Mean
    Normalize
  ]
]

In [19]:
def tripletLoss(v1,v2,alpha = 0.25):
  v = fastnp.dot(v1,v2.T)
  positives = fastnp.diag(v)
  size = len(positives)
  #mean neg
  x = v * (1- fastnp.eye(size))
  mean_neg = fastnp.sum(x,axis=1,keepdims = True)/(size-1)
  l1 = fastnp.maximum((mean_neg - positives + alpha) , 0)
  # closest neg
  y = v - fastnp.eye(positives.shape[0])*2
  close_neg = y.max(axis=1)
  l2 = fastnp.maximum(close_neg-positives+alpha,0)
  total_loss = fastnp.sum(l1+l1)/2
  return total_loss


In [20]:
v1 = fastnp.array([1,-0.2,0.9,-1]).reshape(-1,1)
v2 = fastnp.array([0.9,-0.3,0.3,-0.9]).reshape(-1,1)
tripletLoss(v1,v2)


DeviceArray(0.26999998, dtype=float32)

In [21]:
from functools import partial

def TripletLoss(alpha = 0.25):
  tripletLoss_fn = partial(tripletLoss,alpha = alpha)
  return tl.Fn('TripletLoss',tripletLoss_fn)

In [22]:
lr_schedule = trax.lr.warmup_and_rsqrt_decay(400, 0.01)
def train_model(model,train_generator,eval_generator,n_steps = 1,output_dir = "/DQ"):
  train_task = training.TrainTask(train_generator,TripletLoss(),trax.optimizers.Adam(0.05),lr_schedule=lr_schedule)
  eval_task = training.EvalTask(eval_generator,[TripletLoss()])
  loop = training.Loop(model,train_task,eval_tasks=[eval_task],output_dir=output_dir)
  loop.run(n_steps=n_steps)
  return loop

In [24]:
train_loop = train_model(siamese_model(),train_generator,eval_generator,12000)


Step  12100: Ran 100 train steps in 51.47 secs
Step  12100: train TripletLoss |  0.33140698
Step  12100: eval  TripletLoss |  4.76760387

Step  12200: Ran 100 train steps in 16.66 secs
Step  12200: train TripletLoss |  0.31425154
Step  12200: eval  TripletLoss |  2.20781422

Step  12300: Ran 100 train steps in 12.69 secs
Step  12300: train TripletLoss |  0.26636308
Step  12300: eval  TripletLoss |  2.66634417

Step  12400: Ran 100 train steps in 10.22 secs
Step  12400: train TripletLoss |  0.12685496
Step  12400: eval  TripletLoss |  1.17061698

Step  12500: Ran 100 train steps in 10.63 secs
Step  12500: train TripletLoss |  0.38533279
Step  12500: eval  TripletLoss |  10.50728130

Step  12600: Ran 100 train steps in 11.01 secs
Step  12600: train TripletLoss |  0.32286716
Step  12600: eval  TripletLoss |  0.04778004

Step  12700: Ran 100 train steps in 11.26 secs
Step  12700: train TripletLoss |  0.48639047
Step  12700: eval  TripletLoss |  1.31389308

Step  12800: Ran 100 train steps

In [25]:
model = siamese_model()
model.init_from_file('/DQ/model.pkl.gz')
#model.init_from_file('model.pkl.gz')

In [26]:
df_test = df[['question1','question2']]
y_test = df.is_duplicate
test_x = list(df_test.question1)
test_y = list(df_test.question2)
test_label = fastnp.array(y_test.to_numpy())

In [27]:
len(test_x)

404351

In [28]:
#test
arr1,arr2 = next(dataGenerator(test_x,test_y,1,vocab['<PAD>'],shuffle=False))
v1,v2 = model((arr1,arr2))
d = fastnp.dot(v1,v2.T)
print(d)

[[0.9226856]]


In [29]:
def compute_accuracy(test_x,test_y,test_label,model,batch_size,vocab,threshold):
  iter = int(len(test_x)/batch_size)
  acc = 0
  for i in range(0,len(test_x),batch_size):
    label = list(test_label[i:i+batch_size])
    arr1,arr2 = next(dataGenerator(test_x,test_y,batch_size,vocab['<PAD>'],shuffle=False))
    v1,v2 = model((arr1,arr2))
    for j in range(batch_size):
      d = fastnp.dot(v1[j],v2[j].T)
      pred = (d>threshold).astype(int)
      correct = (pred == label[j]).astype(int)
      acc += correct
  accuracy = acc/len(test_x)
  return accuracy


In [ ]:
acc = compute_accuracy(test_x,test_y,test_label,model,64,vocab,0.7)
acc

In [37]:
def predict(q1,q2,model,dataGenerator,vocab,threshold=0.8):
  arr1,arr2 = next(dataGenerator(q1,q2,1,vocab['<PAD>'],shuffle=False))
  v1,v2 = model((arr1,arr2))
  d = fastnp.dot(v1,v2.T)
  print(d)
  result = (d>threshold)
  return result

In [38]:
q1 = "When will I see you?"
q2 = "When can I see you again?"
print(predict(q1,q2,model,dataGenerator,vocab))

[[0.9999999]]
[[ True]]


In [ ]:
# Feel free to try with your own questions
q1 = "Do they enjoy eating the dessert?"
q2 = "Do they like hiking in the desert?"
# 1 means it is duplicated, 0 otherwise
predict(q1 , q2, model,dataGenerator, vocab)